In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Preprocessing

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN","NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = []
app_types_counts = application_df["APPLICATION_TYPE"].value_counts()
app_types_names = app_types_counts.index
n = 0

for app_type_count in app_types_counts:
  if app_type_count<=10:
    app_type = app_types_names[n]
    application_types_to_replace.append(app_type)
  n +=1

print(application_types_to_replace)

['T25', 'T14', 'T29', 'T15', 'T17']


In [6]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_value_counts = application_df["CLASSIFICATION"].value_counts()
class_value_names = class_value_counts.index
n=0

for classification_count in class_value_counts:
  if classification_count>1:
    print(f'{class_value_names[n]} - {classification_count}')
  n+=1

C1000 - 17326
C2000 - 6074
C1200 - 4837
C3000 - 1918
C2100 - 1883
C7000 - 777
C1700 - 287
C4000 - 194
C5000 - 116
C1270 - 114
C2700 - 104
C2800 - 95
C7100 - 75
C1300 - 58
C1280 - 50
C1230 - 36
C1400 - 34
C7200 - 32
C2300 - 32
C1240 - 30
C8000 - 20
C7120 - 18
C1500 - 16
C1800 - 15
C6000 - 15
C1250 - 14
C8200 - 11
C1238 - 10
C1278 - 10
C1235 - 9
C1237 - 9
C7210 - 7
C2400 - 6
C1720 - 6
C4100 - 6
C1257 - 5
C1600 - 5
C1260 - 3
C2710 - 3
C0 - 3
C3200 - 2
C1234 - 2
C1246 - 2
C1267 - 2
C1256 - 2


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = []
class_types_counts = application_df["CLASSIFICATION"].value_counts()
class_types_names = class_types_counts.index
n = 0

for class_type_count in class_types_counts:
  if class_type_count<=10:
    class_type = class_types_names[n]
    classifications_to_replace.append(class_type)
  n +=1

print(classifications_to_replace)

['C1238', 'C1278', 'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150']


In [10]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
Other      118
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [11]:
# Check replacemnts worked for applications
application_df["APPLICATION_TYPE"].unique()

array(['T10', 'T3', 'T5', 'T7', 'T4', 'T6', 'T2', 'T9', 'T19', 'T8',
       'T13', 'T12', 'Other'], dtype=object)

In [12]:
# Check replacements worked for categoreies
application_df["CLASSIFICATION"].unique()

array(['C1000', 'C2000', 'C3000', 'C1200', 'C2700', 'C7000', 'C7200',
       'C1700', 'C4000', 'C7100', 'C2800', 'C6000', 'C2100', 'Other',
       'C5000', 'C7120', 'C1800', 'C1400', 'C1270', 'C2300', 'C8200',
       'C1500', 'C1300', 'C1230', 'C1280', 'C1240', 'C1250', 'C8000'],
      dtype=object)

In [13]:
# Display dataframe
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df)
application_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0


In [15]:
# Split our preprocessed data into our features and target arrays
features_array = application_dummies_df.drop(columns=["IS_SUCCESSFUL"]).values

target_array = (application_dummies_df["IS_SUCCESSFUL"]).values


In [16]:
#Diplay our split target array
target_array

array([1, 1, 0, ..., 0, 1, 0])

In [17]:
# Display our split features array
features_array

array([[       1,     5000,        0, ...,        0,        1,        0],
       [       1,   108590,        0, ...,        0,        1,        0],
       [       1,     5000,        0, ...,        0,        1,        0],
       ...,
       [       1,     5000,        0, ...,        0,        1,        0],
       [       1,     5000,        0, ...,        0,        1,        0],
       [       1, 36500179,        0, ...,        0,        1,        0]])

In [21]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features_array,target_array,random_state=1)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
X_train_scaled

array([[ 0.01079982, -0.02957055, -0.01870803, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.02957055, -0.01870803, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.02957055, -0.01870803, ..., -0.07529085,
         0.02925687, -0.02925687],
       ...,
       [ 0.01079982, -0.02957055, -0.01870803, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.02957055, -0.01870803, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.02957055, -0.01870803, ..., -0.07529085,
         0.02925687, -0.02925687]])

In [24]:
y_train

array([0, 1, 1, ..., 0, 0, 0])

## Compile, Train and Evaluate the Model

In [25]:
# Count features
features_count = len(X_train_scaled[0])
features_count


69

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=4, activation="relu", input_dim=features_count))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 280       
                                                                 
 dense_1 (Dense)             (None, 2)                 10        
                                                                 
 dense_2 (Dense)             (None, 1)                 3         
                                                                 
Total params: 293 (1.14 KB)
Trainable params: 293 (1.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

In [32]:
# Train the model
fit_nn = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.6186 - accuracy: 0.7040
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5925 - accuracy: 0.7114
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5854 - accuracy: 0.7125
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5811 - accuracy: 0.7154
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5786 - accuracy: 0.7179
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5763 - accuracy: 0.7195
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5737 - accuracy: 0.7211
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5714 - accuracy: 0.7216
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5695 - accuracy: 0.7220
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5680 - accura

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5695 - accuracy: 0.7214 - 335ms/epoch - 1ms/step
Loss: 0.5694850087165833, Accuracy: 0.7213994264602661


In [38]:
# Export our model to HDF5 file
nn.save("/content/drive/MyDrive/Colab Notebooks/applicatn_sucess_model1.HDF5")